# Exploration der Dataset
In diesem Notebook werden die ersten Schritte im Data Wrangling gemacht, das explorieren. 
Da die Datasets aber sehr gross sind, wird DuckDB genutzt, um die Daten erstmal in Memory zu persistieren, 
damit Stück für Stück die Daten eingelesen werden können.

In [ ]:
# Imports
import duckdb
import pandas as pd
import re

## Einlesen der Daten in DuckDB

**DuckDB** ist eine in-memory Datenbank, welche uns ermöglicht einen sehr grossen Datensatz einzulesen und diesen Stückweise
im Code zu verwenden.

In [ ]:
# Einlesen der CSV-Datei und Konvertierung in Parquet
con = duckdb.connect("mydb.duckdb")
data_sets = [
     "data/311-service-requests-from-2010-to-present.csv",
     "data/affordable_housing.csv",
]

for csv_file in data_sets:
    # Filename für Parquet bestimmen und Tabelle benennen 
    parquet_file = csv_file.replace(".csv", ".parquet")
    table_name = csv_file.split("/")[-1].replace(".csv", "").replace("-", "_")
    # Tabellenname darf nicht mit Zahl beginnen
    if re.match(r'^\d', table_name):
        table_name = "t_" + table_name

    print(f"Tablename: {table_name}")
    try:
        con.execute(f"""
            COPY (SELECT * FROM read_csv_auto('{csv_file}'))
            TO '{parquet_file}' (FORMAT 'parquet');
        """)
        print(f"✅ {csv_file} → {parquet_file}")
        print("--------------------------------")
        con.execute(f"""
            CREATE OR REPLACE TABLE {table_name} AS
            SELECT * FROM read_parquet('{parquet_file}');
        """)
        print(f"✅ {csv_file} erfolgreich konvertiert!")
        print("--------------------------------")
        print("--------------------------------")
    except Exception as e:
        print("❌ Fehler bei der Konvertierung:", e)

print("✅ CSV erfolgreich nach Parquet konvertiert!")
con.close()

In [ ]:
# Aufbau der Verbindung zur DuckDB-Datenbank
con = duckdb.connect("mydb.duckdb")

# 5 Zeilen aus CSV
df_311 = con.execute("""
    SELECT *
    FROM t_311_service_requests_from_2010_to_present
    WHERE "Closed Date" IS NOT NULL
    LIMIT 10
""").fetchdf()

df_affordable = con.execute("""
    SELECT *
    FROM affordable_housing
    LIMIT 10
""").fetchdf()

df_affordable.head()